Vom analiza ce subiecte au fost votate cel mai similar de către toate partidele.

© 2019 Dan Gheorghe Haiduc - Licență [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/)<br />
Distribuirea și modificarea sunt permise numai cu condiția atribuirii autorului.<br />

# Încărcare date

In [2]:
%matplotlib inline

import pandas as pd
pd.options.display.max_rows = 100

# Date obtinute de la Parlament Transparent, Popescu, Marina și Chiru, Mihail.
# https://parlament.openpolitics.ro/export/
# Datele pot fi reutilizate sub licență CC-BY-SA-4.0 sau CC-BY-SA-3.0-RO
# Datele se refera doar la Camera Deputatilor (nu si Senat).

!wget -nc https://parlament.openpolitics.ro/export/membri_grupuri.csv
!wget -nc https://parlament.openpolitics.ro/export/voturi.csv

File ‘membri_grupuri.csv’ already there; not retrieving.

File ‘voturi.csv’ already there; not retrieving.



In [3]:
# Ordonare obiectivă: putere (PSD) -> opoziție (partide nesimilare cu PSD)
coloane_partide = ['PSD', 'ALDE', 'Minorități', 'UDMR', 'Neafiliați', 'PNL', 'PMP', 'USR']
descriere_partide = [
    'Grupul parlamentar al Partidului Social Democrat',
    'Grupul parlamentar ALDE (Alianţa liberalilor şi democraţilor)',
    'Grupul parlamentar al minorităţilor naţionale',
    'Grupul parlamentar al Uniunii Democrate Maghiare din România',
    'Deputati neafiliati',
    'Grupul parlamentar al Partidului Naţional Liberal',
    'Grupul parlamentar al Partidului Mişcarea Populară',
    'Grupul parlamentar al Uniunii Salvaţi România',
]

membri = pd.read_csv('membri_grupuri.csv')

for partid, abrev in zip(descriere_partide, coloane_partide):
    membri = membri.replace(partid, abrev)

voturi = pd.read_csv('voturi.csv')
voturi_cu_partide = voturi.merge(membri[['nume', 'partid']])
voturi_cu_partide['partid'].fillna('niciunul curent', inplace=True)
voturi_cu_partide

,data,cod cdep,nume,vot,vot grup,partid
0,2017-02-14,16105,Antoneta Ioniţă,da,da,PNL
1,2017-02-14,16106,Antoneta Ioniţă,da,da,PNL
2,2017-02-14,16107,Antoneta Ioniţă,da,da,PNL
3,2017-02-14,16108,Antoneta Ioniţă,nu,nu,PNL
4,2017-02-14,16109,Antoneta Ioniţă,da,da,PNL
...,...,...,...,...,...,...
415760,2019-10-22,22897,Andrian Ampleev,da,da,Minorități
415761,2019-10-22,22898,Andrian Ampleev,da,da,Minorități
415762,2019-10-22,22899,Andrian Ampleev,da,da,Minorități
415763,2019-10-22,22900,Andrian Ampleev,da,da,Minorități


# Procesare

In [4]:
grupate = voturi_cu_partide.groupby(['cod cdep', 'vot', 'partid'], as_index=False).count().sort_values(['cod cdep', 'vot', 'partid'])
grupate = grupate.pivot_table(index='cod cdep', columns=['partid', 'vot'], values=['data'])['data'].fillna(0)

In [29]:
def normalize():
    vot_relativ = grupate.copy()
    sume = {}
    for p in coloane_partide:
        total_voturi = vot_relativ[p].sum(axis=1)
        vot_relativ.loc[:, p] = vot_relativ.loc[:, p].divide(total_voturi, axis=0).values
    return vot_relativ

from IPython.core.display import display, HTML

def detalii_vot(cod_cdep, scor):
    display(HTML(f"<a href=http://www.cdep.ro/pls/steno/evot2015.nominal?idv={cod_cdep}>Vot {cod_cdep}</a>: scor {scor:.3f} "))

def serie_link_cdep(serie, nr_top=10):
    top = serie.sort_values(ascending=False).head(nr_top)
    for cod, scor in zip(top.index, top):
        detalii_vot(cod, scor)

deviatie_medie = normalize().std(axis=1, level=1).mean(axis=1)
        
display(HTML('Voturi cu deviație mare'))
serie_link_cdep(deviatie_medie, 20)

display(HTML('Voturi cu deviație mică'))
serie_link_cdep(-deviatie_medie, 250)


# Similaritate între voturile partidelor

Ce partide votează similar pentru aceleași subiecte?

In [6]:
def diferente(p1, p2):
    vot_relativ = normalize()
    return abs(vot_relativ[p1] - vot_relativ[p2]).sum(axis=1)

def sim(p1, p2):
    return 1-(diferente(p1, p2)/2).mean()

sim_mat = pd.DataFrame(0, index=coloane_partide, columns=coloane_partide)
for p1 in coloane_partide:
    for p2 in coloane_partide:
        if not sim_mat.loc[p1, p2]:
            s = sim(p1, p2)
            sim_mat.loc[p1, p2] = s
            sim_mat.loc[p2, p1] = s

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [6, 6]

import seaborn as sns
sns.set()
ax = sns.heatmap(sim_mat, square=True)

# https://stackoverflow.com/questions/56942670/matplotlib-seaborn-first-and-last-row-cut-in-half-of-heatmap-plot
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
ax.set_title('Similaritate între voturile partidelor')

# Ce înseamnă toate acestea?
* Partidele sunt destul de similare (de exemplu, PSD și USR au votat similar la aprox. 70% din voturi).
* Avem de unde alege (există diferențe importante în acele 30% rămase).
* Urmăriți acest blog pentru partea a 2-a!